In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
!pip install transformers
!pip install torchvision
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.7 MB/s eta 0:00:00


In [ ]:
!unzip "/content/drive/MyDrive/cv_dataset.zip" -d "/content/cv_dataset"


Archive:  /content/drive/MyDrive/cv_dataset.zip
   creating: /content/cv_dataset/cv_dataset/
  inflating: /content/cv_dataset/__MACOSX/._cv_dataset  
  inflating: /content/cv_dataset/cv_dataset/.DS_Store  
  inflating: /content/cv_dataset/__MACOSX/cv_dataset/._.DS_Store  
   creating: /content/cv_dataset/cv_dataset/yes_scaffolding/
   creating: /content/cv_dataset/cv_dataset/no_scaffolding/
  inflating: /content/cv_dataset/cv_dataset/yes_scaffolding/yes_scaffolding00257.png  
  inflating: /content/cv_dataset/cv_dataset/yes_scaffolding/yes_scaffolding00243.png  
  inflating: /content/cv_dataset/cv_dataset/yes_scaffolding/yes_scaffolding00055.png  
  inflating: /content/cv_dataset/cv_dataset/yes_scaffolding/yes_scaffolding00041.png  
  inflating: /content/cv_dataset/cv_dataset/yes_scaffolding/yes_scaffolding00069.png  
  inflating: /content/cv_dataset/cv_dataset/yes_scaffolding/yes_scaffolding00096.png  
  inflating: /content/cv_dataset/cv_dataset/yes_scaffolding/yes_scaffolding00082.png

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import timm
from torch import nn, optim

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
}

main_dir = '/content/cv_dataset/cv_dataset'
dataset = datasets.ImageFolder(main_dir, transform=data_transforms['train'])

# Splitting dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
# EfficientNet model
model = timm.create_model("efficientnet_b3", pretrained=True, num_classes=1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2.12e-3)

# Training
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs.view(-1), labels.float())

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")

Epoch 1/5, Loss: 2.8737454059578122
Epoch 2/5, Loss: 1.1625414285677143
Epoch 3/5, Loss: 0.41371855461260393
Epoch 4/5, Loss: 0.3174331374466419
Epoch 5/5, Loss: 0.1481312752834388


In [ ]:
# Calculate accuracy
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

model.eval()

total_loss, total_acc = 0, 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs.view(-1), labels.float())

        acc = binary_accuracy(outputs.view(-1), labels.float())

        total_loss += loss.item()
        total_acc += acc.item()

# Overall average loss and accuracy
avg_loss = total_loss / len(val_loader)
avg_acc = total_acc / len(val_loader)

print(f"Validation Loss: {avg_loss:.4f}")
print(f"Validation Accuracy: {avg_acc*100:.2f}%")

import os

# Ensure the 'Good models' directory exists
if not os.path.exists('Good models'):
    os.makedirs('Good models')

# Function to get the next available model index
def get_next_model_index(directory):
    existing_files = os.listdir(directory)
    existing_indices = [int(f.split('_')[-1].split('.')[0]) for f in existing_files if f.startswith('model_') and f.endswith('.pth')]
    if existing_indices:
        return max(existing_indices) + 1
    return 1  # Return 1 if no existing models

# Check the accuracy
if avg_acc * 100 > 90:
    next_index = get_next_model_index('Good models')
    model_path = os.path.join('Good models', f'model_{next_index}.pth')
    torch.save(model.state_dict(), model_path)
    print(f"Model saved as {model_path}")


Validation Loss: 0.0981
Validation Accuracy: 96.88%
Model saved as Good models/model_5.pth


In [ ]:
import os
import numpy as np
from PIL import Image

root_dir = 'predictions'
sub_dirs = ['correct_yes', 'correct_no', 'incorrect_yes', 'incorrect_no']
for sub_dir in sub_dirs:
    os.makedirs(os.path.join(root_dir, sub_dir), exist_ok=True)

model.eval()

counters = {sub_dir: 0 for sub_dir in sub_dirs}
correct_yes, correct_no, incorrect_yes, incorrect_no = 0, 0, 0, 0

with torch.no_grad():
    for image, label in val_dataset:
        image_tensor = image.unsqueeze(0).to(device)

        output = model(image_tensor)
        predicted = torch.sigmoid(output).item() > 0.5

        if label == 1 and predicted:
            correct_yes += 1
            folder = 'correct_yes'
        elif label == 0 and not predicted:
            correct_no += 1
            folder = 'correct_no'
        elif label == 1 and not predicted:
            incorrect_no += 1
            folder = 'incorrect_no'
        else:
            incorrect_yes += 1
            folder = 'incorrect_yes'

        counters[folder] += 1
        img_name = f"{folder}_{counters[folder]:03}.png"
        path = os.path.join(root_dir, folder, img_name)
        Image.fromarray((image.numpy().transpose(1, 2, 0) * 255).astype(np.uint8)).save(path)

total = len(val_dataset)

yes_accuracy = (correct_yes / (correct_yes + incorrect_yes)) * 100
no_accuracy = (correct_no / (correct_no + incorrect_no)) * 100

with open(os.path.join(root_dir, 'metrics.txt'), 'w') as file:
    file.write(f"Yes Accuracy: {yes_accuracy:.2f}%\n")
    file.write(f"No Accuracy: {no_accuracy:.2f}%\n")






In [ ]:

PATH = "trained_model.pth"

torch.save(model.state_dict(), PATH)

print(f"Model's state_dict saved to {PATH}")


Model's state_dict saved to trained_model.pth


In [ ]:
import os
import numpy as np
from PIL import Image
import torch

# Function to load all saved models from the 'Good models' directory
def load_saved_models(directory, device):
    model_paths = [os.path.join(directory, fname) for fname in os.listdir(directory) if fname.endswith('.pth')]
    models = []
    for model_path in model_paths:
        model_instance = timm.create_model("efficientnet_b3", pretrained=False, num_classes=1).to(device)
        model_instance.load_state_dict(torch.load(model_path))
        model_instance.eval()
        models.append(model_instance)
    return models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
models = load_saved_models('Good models', device)

root_dir = 'predictions'
sub_dirs = ['correct_yes', 'correct_no', 'incorrect_yes', 'incorrect_no']
for sub_dir in sub_dirs:
    os.makedirs(os.path.join(root_dir, sub_dir), exist_ok=True)

counters = {sub_dir: 0 for sub_dir in sub_dirs}
correct_yes, correct_no, incorrect_yes, incorrect_no = 0, 0, 0, 0

with torch.no_grad():
    for image, label in val_dataset:
        image_tensor = image.unsqueeze(0).to(device)
        predictions = []

        # Collect predictions from all models
        for model in models:
            output = model(image_tensor)
            predicted = torch.sigmoid(output).item() > 0.5
            predictions.append(predicted)

        # Requiring consensus among at least 5 of 5 models
        if predictions.count(True) >= 5:
            consensus_prediction = 1
        else:
            consensus_prediction = 0

        if label == 1 and consensus_prediction:
            correct_yes += 1
            folder = 'correct_yes'
        elif label == 0 and not consensus_prediction:
            correct_no += 1
            folder = 'correct_no'
        elif label == 1 and not consensus_prediction:
            incorrect_no += 1
            folder = 'incorrect_no'
        else:
            incorrect_yes += 1
            folder = 'incorrect_yes'

        counters[folder] += 1
        img_name = f"{folder}_{counters[folder]:03}.png"
        path = os.path.join(root_dir, folder, img_name)
        Image.fromarray((image.numpy().transpose(1, 2, 0) * 255).astype(np.uint8)).save(path)

total = len(val_dataset)

yes_accuracy = (correct_yes / (correct_yes + incorrect_yes)) * 100
no_accuracy = (correct_no / (correct_no + incorrect_no)) * 100

with open(os.path.join(root_dir, 'metrics.txt'), 'w') as file:
    file.write(f"Yes Accuracy: {yes_accuracy:.2f}%\n")
    file.write(f"No Accuracy: {no_accuracy:.2f}%\n")

print(f"True Positives: {correct_yes}")
print(f"True Negatives: {correct_no}")
print(f"False Positives: {incorrect_yes}")
print(f"False Negatives: {incorrect_no}")


True Positives: 32
True Negatives: 40
False Positives: 0
False Negatives: 10
